## Production des données aggrégées à partir avec du SQL

In [0]:
%python
from pyspark.sql.functions import count, countDistinct, sum, col

In [0]:
%run "../set-up/mount_data_route"

In [0]:
%run "../utils/create_frequency_table_fn"

In [0]:
%python
credits_df_cleaned = spark.read.parquet(f"{silver_folder_path}/credit_card_data")

In [0]:
%python
# Nombre de lignes total
credits_df_cleaned.select(count("Y")).show()
# credits_df_cleaned.select(countDistinct("Y")).show()

In [0]:
%python
# Nombre 
display(create_frequency_table(credits_df_cleaned, "Y"))

In [0]:
CREATE DATABASE IF NOT EXISTS visualization_db;
USE visualization_db;

In [0]:
%python
credits_df_cleaned.write.format("parquet").saveAsTable("visualization_db.credit_data")

In [0]:
SHOW DATABASES;

In [0]:
DESCRIBE DATABASE visualization_db;

In [0]:
DESCRIBE DATABASE EXTENDED visualization_db;

In [0]:
SHOW TABLES IN visualization_db;

In [0]:
DESC EXTENDED credit_data;

In [0]:
/* Statistques descriptives sur les montants */
SELECT 
    COUNT(*) as total_transactions,
    COUNT(DISTINCT Y) as classes,
    MIN(Amount) as min_amount,
    MAX(Amount) as max_amount,
    ROUND(AVG(Amount), 2) as avg_amount,
    ROUND(STDDEV(Amount), 2) as std_amount,
    PERCENTILE_APPROX(Amount, 0.5) as median_amount
FROM credit_data

In [0]:
/* Distribution des classes */
SELECT 
    CASE 
        WHEN Y = 0 THEN 'Normal'
        WHEN Y = 1 THEN 'Fraude'
    END as transaction_type,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
FROM credit_data
GROUP BY Y
ORDER BY Y

In [0]:
/* Distribution des montants par classe */
SELECT 
    CASE 
        WHEN Y = 0 THEN 'Normal'
        WHEN Y = 1 THEN 'Fraude'
    END as transaction_type,
    COUNT(*) as count,
    ROUND(MIN(Amount), 2) as min_amount,
    ROUND(MAX(Amount), 2) as max_amount,
    ROUND(AVG(Amount), 2) as avg_amount,
    ROUND(STDDEV(Amount), 2) as std_amount,
    PERCENTILE_APPROX(Amount, 0.25) as Q1,
    PERCENTILE_APPROX(Amount, 0.5) as median,
    PERCENTILE_APPROX(Amount, 0.75) as Q3
FROM credit_data
GROUP BY Y
ORDER BY Y